# JS: Core

> Master composer for card stack JavaScript. Combines viewport height,
> scroll navigation, page navigation, width/scale/count management, and
> the master HTMX coordinator into a single namespaced IIFE.

In [ ]:
#| default_exp js.core

In [ ]:
#| export
from typing import Any, Optional, Tuple

from fasthtml.common import Script

from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.core.models import CardStackUrls, CardStackState
from cjm_fasthtml_card_stack.core.constants import (
    width_storage_key, scale_storage_key, card_count_storage_key,
    auto_count_storage_key,
    DEFAULT_CARD_WIDTH, DEFAULT_CARD_SCALE, DEFAULT_VISIBLE_COUNT,
)
from cjm_fasthtml_card_stack.js.viewport import generate_viewport_height_js
from cjm_fasthtml_card_stack.js.scroll import generate_scroll_nav_js
from cjm_fasthtml_card_stack.js.navigation import generate_page_nav_js

## Width / Scale / Card Count Management

These JS fragments handle localStorage persistence and live updates for
the three user-adjustable viewport preferences.

In [ ]:
#| export
def _generate_width_mgmt_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config with slider bounds
    urls: CardStackUrls,  # URL bundle (save_width)
) -> str:  # JS code fragment for width management
    """Generate JS for width slider management."""
    storage_key = width_storage_key(config.prefix)
    return f"""
        // === Width Management ===
        const _WIDTH_KEY = '{storage_key}';
        let _saveWidthTimer = null;

        function _saveWidthToServer(val) {{
            if (!'{urls.save_width}') return;
            clearTimeout(_saveWidthTimer);
            _saveWidthTimer = setTimeout(function() {{
                htmx.ajax('POST', '{urls.save_width}', {{
                    swap: 'none', values: {{ card_width: val }}
                }});
            }}, 500);
        }}

        ns.updateWidth = function(value) {{
            const inner = document.getElementById('{ids.card_stack_inner}');
            if (!inner) return;
            inner.style.maxWidth = value + 'rem';
            try {{ localStorage.setItem(_WIDTH_KEY, value); }} catch (e) {{}}
            const slider = document.getElementById('{ids.width_slider}');
            if (slider && parseInt(slider.value) !== parseInt(value)) slider.value = value;
            _saveWidthToServer(value);
            if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
        }};

        ns.decreaseWidth = function() {{
            const slider = document.getElementById('{ids.width_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_WIDTH};
            ns.updateWidth(Math.max({config.card_width_min}, current - {config.card_width_step}));
        }};

        ns.increaseWidth = function() {{
            const slider = document.getElementById('{ids.width_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_WIDTH};
            ns.updateWidth(Math.min({config.card_width_max}, current + {config.card_width_step}));
        }};

        ns.applyWidth = function() {{
            const inner = document.getElementById('{ids.card_stack_inner}');
            if (!inner) return;
            let val = {DEFAULT_CARD_WIDTH};
            try {{ const s = localStorage.getItem(_WIDTH_KEY); if (s) val = parseInt(s); }} catch (e) {{}}
            inner.style.maxWidth = val + 'rem';
            const slider = document.getElementById('{ids.width_slider}');
            if (slider) slider.value = val;
        }};
    """

In [ ]:
#| export
def _generate_scale_mgmt_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config with slider bounds
    urls: CardStackUrls,  # URL bundle (save_scale)
) -> str:  # JS code fragment for scale management
    """Generate JS for scale slider management."""
    storage_key = scale_storage_key(config.prefix)
    return f"""
        // === Scale Management ===
        const _SCALE_KEY = '{storage_key}';
        let _saveScaleTimer = null;

        function _saveScaleToServer(val) {{
            if (!'{urls.save_scale}') return;
            clearTimeout(_saveScaleTimer);
            _saveScaleTimer = setTimeout(function() {{
                htmx.ajax('POST', '{urls.save_scale}', {{
                    swap: 'none', values: {{ card_scale: val }}
                }});
            }}, 500);
        }}

        function _applyScaleCssProperty(val) {{
            const cs = document.getElementById('{ids.card_stack}');
            if (cs) cs.style.setProperty('--card-stack-scale', val);
        }}

        ns.updateScale = function(value) {{
            _applyScaleCssProperty(value);
            try {{ localStorage.setItem(_SCALE_KEY, value); }} catch (e) {{}}
            const slider = document.getElementById('{ids.scale_slider}');
            if (slider && parseInt(slider.value) !== parseInt(value)) slider.value = value;
            _saveScaleToServer(value);
            if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
        }};

        ns.decreaseScale = function() {{
            const slider = document.getElementById('{ids.scale_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_SCALE};
            ns.updateScale(Math.max({config.card_scale_min}, current - {config.card_scale_step}));
        }};

        ns.increaseScale = function() {{
            const slider = document.getElementById('{ids.scale_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_SCALE};
            ns.updateScale(Math.min({config.card_scale_max}, current + {config.card_scale_step}));
        }};

        ns.applyScale = function() {{
            let val = {DEFAULT_CARD_SCALE};
            try {{ const s = localStorage.getItem(_SCALE_KEY); if (s) val = parseInt(s); }} catch (e) {{}}
            _applyScaleCssProperty(val);
            const slider = document.getElementById('{ids.scale_slider}');
            if (slider) slider.value = val;
        }};
    """

In [ ]:
#| export
def _generate_card_count_mgmt_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config with count options
    urls: CardStackUrls,  # URL bundle (update_viewport)
) -> str:  # JS code fragment for card count management
    """Generate JS for card count selector management."""
    storage_key = card_count_storage_key(config.prefix)
    auto_key = auto_count_storage_key(config.prefix)
    valid_counts = ', '.join(str(c) for c in config.visible_count_options)
    return f"""
        // === Card Count Management ===
        const _COUNT_KEY = '{storage_key}';
        const _AUTO_KEY = '{auto_key}';
        const _VALID_COUNTS = [{valid_counts}];

        function _isAutoMode() {{
            try {{ return localStorage.getItem(_AUTO_KEY) === 'true'; }} catch (e) {{ return false; }}
        }}

        function _getStoredCount() {{
            try {{
                const s = localStorage.getItem(_COUNT_KEY);
                if (s) {{ const c = parseInt(s); if (_VALID_COUNTS.includes(c)) return c; }}
            }} catch (e) {{}}
            return {DEFAULT_VISIBLE_COUNT};
        }}

        ns.updateCardCount = function(value) {{
            const count = parseInt(value);
            if (!_VALID_COUNTS.includes(count)) return;
            try {{ localStorage.setItem(_COUNT_KEY, count); }} catch (e) {{}}
            const cardStack = document.getElementById('{ids.card_stack}');
            if (cardStack) cardStack.dataset.visibleCount = count;
            if ('{urls.update_viewport}') {{
                htmx.ajax('POST', '{urls.update_viewport}', {{
                    target: '#' + '{ids.card_stack}',
                    swap: 'none',
                    values: {{ visible_count: count }}
                }});
            }}
        }};

        ns._autoUpdateCount = function(count) {{
            // Like updateCardCount but bypasses _VALID_COUNTS validation.
            // Used by auto-adjustment which can set any count.
            const c = Math.max(1, Math.round(count));
            const cardStack = document.getElementById('{ids.card_stack}');
            if (cardStack) cardStack.dataset.visibleCount = c;
            if ('{urls.update_viewport}') {{
                htmx.ajax('POST', '{urls.update_viewport}', {{
                    target: '#' + '{ids.card_stack}',
                    swap: 'none',
                    values: {{ visible_count: c }}
                }});
            }}
        }};

        ns.handleCountChange = function(value) {{
            // Entry point for dropdown onchange — handles both "auto" and numeric values.
            if (value === 'auto') {{
                try {{ localStorage.setItem(_AUTO_KEY, 'true'); }} catch (e) {{}}
                if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
            }} else {{
                try {{ localStorage.removeItem(_AUTO_KEY); }} catch (e) {{}}
                ns.updateCardCount(parseInt(value));
            }}
        }};

        function _syncCountDropdown() {{
            const sel = document.getElementById('{ids.card_count_select}');
            if (!sel) return;
            if (_isAutoMode()) {{
                if (sel.value !== 'auto') sel.value = 'auto';
            }} else {{
                const stored = _getStoredCount();
                if (parseInt(sel.value) !== stored) sel.value = stored;
            }}
        }}
    """

## Auto Visible Count Adjustment

Overflow-based feedback loop that dynamically determines how many cards
fit in the viewport. Increments by 2 for center focus (keeping symmetry)
or by 1 for top/bottom focus, until one card overflows on each side.

In [ ]:
#| export
def _generate_auto_adjust_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config for auto mode check
    urls: CardStackUrls,  # URL bundle (update_viewport)
    focus_position: Optional[int] = None,  # Focus slot offset (None=center, -1=bottom, 0=top)
) -> str:  # JS code fragment for auto visible count adjustment
    """Generate JS for automatic visible count adjustment based on overflow detection."""
    js_focus_pos = "null" if focus_position is None else str(focus_position)
    return f"""
        // === Auto Visible Count Adjustment ===
        let _autoAdjusting = false;
        let _autoAdjustTimer = null;
        const _AUTO_FOCUS_POS = {js_focus_pos};
        const _AUTO_STEP = (_AUTO_FOCUS_POS === null) ? 2 : 1;

        function _getAutoCurrentCount() {{
            const cs = document.getElementById('{ids.card_stack}');
            return cs ? parseInt(cs.dataset.visibleCount || '{DEFAULT_VISIBLE_COUNT}') : {DEFAULT_VISIBLE_COUNT};
        }}

        function _getAutoTotalItems() {{
            const cs = document.getElementById('{ids.card_stack}');
            return cs ? parseInt(cs.dataset.totalItems || '0') : 0;
        }}

        function _getAutoSectionOverflow() {{
            // Returns max overflow (px) across relevant sections.
            const before = document.getElementById('{ids.viewport_section_before}');
            const after = document.getElementById('{ids.viewport_section_after}');
            let maxOverflow = 0;

            const checkBefore = (_AUTO_FOCUS_POS === null || _AUTO_FOCUS_POS > 0 || _AUTO_FOCUS_POS < 0);
            const checkAfter = (_AUTO_FOCUS_POS === null || _AUTO_FOCUS_POS >= 0);

            if (checkBefore && before) {{
                const o = before.scrollHeight - before.clientHeight;
                if (o > maxOverflow) maxOverflow = o;
            }}
            if (checkAfter && after) {{
                const o = after.scrollHeight - after.clientHeight;
                if (o > maxOverflow) maxOverflow = o;
            }}
            return maxOverflow;
        }}

        function _getAutoAvgCardHeight() {{
            // Average height of rendered viewport-slot elements.
            const cs = document.getElementById('{ids.card_stack}');
            if (!cs) return 100;
            const slots = cs.querySelectorAll('.viewport-slot');
            if (slots.length === 0) return 100;
            let total = 0;
            for (const s of slots) total += s.getBoundingClientRect().height;
            return total / slots.length;
        }}

        function _getAutoGapPx() {{
            // Read computed gap from the before section (or after).
            const section = document.getElementById('{ids.viewport_section_before}')
                         || document.getElementById('{ids.viewport_section_after}');
            if (!section) return 16;
            return parseFloat(getComputedStyle(section).gap) || 16;
        }}

        function _getAutoRemainingSpace() {{
            // Measure remaining space in relevant sections (no overflow case).
            const before = document.getElementById('{ids.viewport_section_before}');
            const after = document.getElementById('{ids.viewport_section_after}');
            let space = 0;

            if (before && before.children.length > 0 &&
                (_AUTO_FOCUS_POS === null || _AUTO_FOCUS_POS > 0 || _AUTO_FOCUS_POS < 0)) {{
                const sRect = before.getBoundingClientRect();
                const first = before.children[0].getBoundingClientRect();
                const gap = first.top - sRect.top;
                if (gap > 0) space += gap;
            }}

            if (after && after.children.length > 0 &&
                (_AUTO_FOCUS_POS === null || _AUTO_FOCUS_POS >= 0)) {{
                const sRect = after.getBoundingClientRect();
                const last = after.children[after.children.length - 1].getBoundingClientRect();
                const gap = sRect.bottom - last.bottom;
                if (gap > 0) space += gap;
            }}

            return space;
        }}

        ns._runAutoAdjust = function() {{
            if (!_isAutoMode() || _autoAdjusting) return;

            const currentCount = _getAutoCurrentCount();
            const totalItems = _getAutoTotalItems();
            if (totalItems === 0) return;

            const overflow = _getAutoSectionOverflow();
            const avgHeight = _getAutoAvgCardHeight();
            const gapPx = _getAutoGapPx();

            if (overflow > 0) {{
                // Overflow exists — check if it's more than ~1 card
                if (overflow > avgHeight * 1.5) {{
                    // Too much overflow — estimate how many to remove
                    const toRemove = Math.ceil((overflow - avgHeight * 0.5) / (avgHeight + gapPx));
                    const adjusted = (_AUTO_FOCUS_POS === null)
                        ? Math.max(_AUTO_STEP, Math.ceil(toRemove / 2) * 2)
                        : Math.max(_AUTO_STEP, toRemove);
                    const newCount = Math.max(1, currentCount - adjusted);
                    if (newCount !== currentCount) {{
                        _autoAdjusting = true;
                        ns._autoUpdateCount(newCount);
                    }}
                }}
                // else: ~1 card overflow = target state, done
            }} else {{
                // No overflow — check if we can add more
                if (currentCount >= totalItems) return;  // All items already visible

                const remaining = _getAutoRemainingSpace();
                const toAdd = Math.floor(remaining / (avgHeight + gapPx));
                const adjusted = (_AUTO_FOCUS_POS === null)
                    ? Math.max(_AUTO_STEP, Math.floor(toAdd / 2) * 2)
                    : Math.max(_AUTO_STEP, toAdd);
                const newCount = Math.min(totalItems, currentCount + adjusted);
                if (newCount > currentCount) {{
                    _autoAdjusting = true;
                    ns._autoUpdateCount(newCount);
                }}
            }}
        }};

        ns.triggerAutoAdjust = function() {{
            // Debounced entry point for external triggers (resize, width, scale).
            if (!_isAutoMode()) return;
            clearTimeout(_autoAdjustTimer);
            _autoAdjustTimer = setTimeout(function() {{
                ns._runAutoAdjust();
            }}, 200);
        }};
    """

## Master Coordinator

Applies all viewport settings (width, height, scroll) then reveals the
viewport using a double-RAF pattern. Also handles HTMX afterSettle events.

In [ ]:
#| export
def _generate_coordinator_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config for prefix-unique listener guards
    focus_position: Optional[int] = None,  # Focus slot offset (None=center, -1=bottom, 0=top)
) -> str:  # JS code fragment for master coordinator
    """Generate JS for the master coordinator and HTMX listener."""
    guard_var = f"_csMasterListener_{config.prefix.replace('-', '_')}"
    js_focus_pos = "null" if focus_position is None else str(focus_position)
    return f"""
        // === Grid Template Management ===
        ns.applyGridTemplate = function() {{
            const inner = document.getElementById('{ids.card_stack_inner}');
            if (!inner) return;
            const focusPosRaw = {js_focus_pos};
            let tmpl;
            if (focusPosRaw === null) {{
                tmpl = '1fr auto 1fr';
            }} else if (focusPosRaw === 0) {{
                tmpl = 'auto 1fr';
            }} else if (focusPosRaw < 0) {{
                tmpl = '1fr auto';
            }} else {{
                tmpl = '1fr auto 1fr';
            }}
            inner.style.gridTemplateRows = tmpl;
        }};

        // === Master Coordinator ===
        ns.applyAllViewportSettings = function() {{
            requestAnimationFrame(function() {{
                if (ns.applyWidth) ns.applyWidth();
                if (ns.applyScale) ns.applyScale();
                if (ns.applyGridTemplate) ns.applyGridTemplate();
                if (ns.recalculateHeight) ns.recalculateHeight();

                const cs = document.getElementById('{ids.card_stack}');
                if (cs) {{
                    cs._scrollNavSetup = false;
                    if (ns._setupScrollNav) ns._setupScrollNav();
                }}

                requestAnimationFrame(function() {{
                    const cs2 = document.getElementById('{ids.card_stack}');
                    if (cs2) cs2.style.opacity = '1';

                    // Continue auto-adjust loop if an adjustment is in flight
                    if (typeof _autoAdjusting !== 'undefined' && _autoAdjusting) {{
                        _autoAdjusting = false;
                        requestAnimationFrame(function() {{
                            if (ns._runAutoAdjust) ns._runAutoAdjust();
                        }});
                    }}
                }});
            }});
        }};

        // === HTMX Event Listener ===
        if (!window.{guard_var}) {{
            window.{guard_var} = true;
            document.body.addEventListener('htmx:afterSettle', function(evt) {{
                const target = evt.detail.target;
                if (!target) return;
                const cs = document.getElementById('{ids.card_stack}');
                const isCSSwap = (
                    target.id === '{ids.card_stack}' ||
                    target.id === '{ids.card_stack_inner}' ||
                    (cs && cs.contains(target))
                );
                if (isCSSwap) {{
                    _syncCountDropdown();
                    ns.applyAllViewportSettings();
                }}
            }});
        }}

        // === Initialize ===
        requestAnimationFrame(function() {{
            _syncCountDropdown();
            setTimeout(function() {{
                ns.applyAllViewportSettings();
                // Trigger auto-adjust after initial layout settles
                if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
            }}, 50);
        }});
    """

## Global Callback Registration

The `cjm-fasthtml-keyboard-navigation` library looks up JS callbacks via
`window[action.jsCallback]()`. Since our functions live on the namespaced
`ns` object, we register global wrappers that delegate to the namespace.

In [ ]:
#| export
# Callback name constants — must match what create_card_stack_nav_actions uses
_GLOBAL_CALLBACKS = (
    "jumpPageUp",
    "jumpPageDown",
    "jumpToFirstItem",
    "jumpToLastItem",
    "decreaseWidth",
    "increaseWidth",
    "decreaseScale",
    "increaseScale",
)

def global_callback_name(
    prefix: str,  # Card stack instance prefix
    callback: str,  # Base callback name (e.g., "jumpPageUp")
) -> str:  # Global function name (e.g., "cs0_jumpPageUp")
    """Generate a prefix-unique global callback name for keyboard navigation."""
    return f"{prefix}_{callback}"

def _generate_global_callbacks_js(
    config: CardStackConfig,  # Config with prefix
) -> str:  # JS code fragment registering global wrappers
    """Register global wrappers for keyboard navigation system."""
    lines = ["        // === Global Keyboard Callbacks ==="]
    for cb in _GLOBAL_CALLBACKS:
        global_name = global_callback_name(config.prefix, cb)
        lines.append(f"        window['{global_name}'] = function() {{ if (ns.{cb}) ns.{cb}(); }};")
    return "\n".join(lines)

## generate_card_stack_js

The main entry point that composes all JS fragments into a single namespaced
IIFE. The consumer calls this once in their step renderer.

In [ ]:
#| export
def generate_card_stack_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    button_ids: CardStackButtonIds,  # Button IDs for keyboard triggers
    config: CardStackConfig,  # Card stack configuration
    urls: CardStackUrls,  # URL bundle for routing
    container_id: str = "",  # Consumer's parent container ID (for height calc)
    extra_scripts: Tuple[str, ...] = (),  # Additional JS to include in the IIFE
    focus_position: Optional[int] = None,  # Focus slot offset (None=center, -1=bottom, 0=top)
) -> Any:  # Script element with all card stack JavaScript
    """Compose all card stack JS into a single namespaced IIFE."""
    prefix = config.prefix
    extra_js = "\n".join(extra_scripts)

    # Collect all fragments
    viewport_js = generate_viewport_height_js(ids, container_id)
    scroll_js = generate_scroll_nav_js(ids, button_ids, config.disable_scroll_in_modes)
    page_nav_js = generate_page_nav_js(button_ids)
    width_js = _generate_width_mgmt_js(ids, config, urls)
    scale_js = _generate_scale_mgmt_js(ids, config, urls)
    count_js = _generate_card_count_mgmt_js(ids, config, urls)
    auto_js = _generate_auto_adjust_js(ids, config, urls, focus_position) if config.auto_visible_count else ""
    global_cbs_js = _generate_global_callbacks_js(config)
    coordinator_js = _generate_coordinator_js(ids, config, focus_position)

    return Script(f"""(function() {{
        window.cardStacks = window.cardStacks || {{}};
        const ns = window.cardStacks['{prefix}'] = {{}};

        {viewport_js}
        {scroll_js}
        {page_nav_js}
        {width_js}
        {scale_js}
        {count_js}
        {auto_js}
        {global_cbs_js}
        {coordinator_js}
        {extra_js}
    }})();""")

In [ ]:
from cjm_fasthtml_card_stack.core.config import _reset_prefix_counter

In [ ]:
# Test generate_card_stack_js produces a Script with all sections
_reset_prefix_counter()
config = CardStackConfig()
ids = CardStackHtmlIds(prefix=config.prefix)
btn = CardStackButtonIds(prefix=config.prefix)
urls = CardStackUrls(
    nav_up="/cs/nav_up", nav_down="/cs/nav_down",
    nav_first="/cs/nav_first", nav_last="/cs/nav_last",
    nav_page_up="/cs/nav_page_up", nav_page_down="/cs/nav_page_down",
    nav_to_index="/cs/nav_to_index",
    update_viewport="/cs/update_viewport",
    save_width="/cs/save_width", save_scale="/cs/save_scale",
)

script = generate_card_stack_js(ids, btn, config, urls, container_id="my-app")
js_text = script.children[0] if script.children else ""

# Check namespace setup
assert "window.cardStacks" in js_text
assert f"'{config.prefix}'" in js_text

# Check all sections present
assert "Viewport Height" in js_text
assert "Scroll Navigation" in js_text
assert "Page Navigation" in js_text
assert "Width Management" in js_text
assert "Scale Management" in js_text
assert "Card Count Management" in js_text
assert "Auto Visible Count Adjustment" in js_text
assert "Grid Template Management" in js_text
assert "Global Keyboard Callbacks" in js_text
assert "Master Coordinator" in js_text
assert "HTMX Event Listener" in js_text

# Check key functions exposed on namespace
assert "ns.updateWidth" in js_text
assert "ns.updateScale" in js_text
assert "ns.updateCardCount" in js_text
assert "ns.handleCountChange" in js_text
assert "ns._autoUpdateCount" in js_text
assert "ns._runAutoAdjust" in js_text
assert "ns.triggerAutoAdjust" in js_text
assert "ns.applyGridTemplate" in js_text
assert "ns.jumpToFirstItem" in js_text
assert "ns.jumpToLastItem" in js_text
assert "ns.applyAllViewportSettings" in js_text
assert "ns.recalculateHeight" in js_text
assert "ns.decreaseWidth" in js_text
assert "ns.increaseWidth" in js_text
assert "ns.decreaseScale" in js_text
assert "ns.increaseScale" in js_text

# Check auto mode functions
assert "_isAutoMode" in js_text
assert "_AUTO_KEY" in js_text
assert "_autoAdjusting" in js_text

# Check auto-adjust triggers in width/scale
assert "ns.triggerAutoAdjust" in js_text

# Check grid template is called in coordinator
assert "if (ns.applyGridTemplate) ns.applyGridTemplate();" in js_text
assert "gridTemplateRows" in js_text

# Check card count uses swap:'none' for OOB updates
assert "swap: 'none',\n                    values: { visible_count:" in js_text

# Check global callback wrappers
prefix = config.prefix
assert f"window['{prefix}_jumpPageUp']" in js_text
assert f"window['{prefix}_jumpToFirstItem']" in js_text
assert f"window['{prefix}_decreaseWidth']" in js_text
assert f"window['{prefix}_increaseWidth']" in js_text
assert f"window['{prefix}_decreaseScale']" in js_text
assert f"window['{prefix}_increaseScale']" in js_text

# Check auto-adjust loop continuation in coordinator
assert "_autoAdjusting" in js_text
assert "ns._runAutoAdjust" in js_text

# Check init triggers auto-adjust
assert "ns.triggerAutoAdjust" in js_text

print("generate_card_stack_js comprehensive tests passed!")

generate_card_stack_js comprehensive tests passed!


In [ ]:
# Test focus_position parameter produces correct JS literals
_reset_prefix_counter()
_fp_config = CardStackConfig()
_fp_ids = CardStackHtmlIds(prefix=_fp_config.prefix)
_fp_btn = CardStackButtonIds(prefix=_fp_config.prefix)

# Default (None) → JS null (center focus)
_fp_script = generate_card_stack_js(_fp_ids, _fp_btn, _fp_config, urls)
_fp_js = _fp_script.children[0] if _fp_script.children else ""
assert "const focusPosRaw = null;" in _fp_js

# Bottom focus (-1)
_fp_script_bottom = generate_card_stack_js(_fp_ids, _fp_btn, _fp_config, urls, focus_position=-1)
_fp_js_bottom = _fp_script_bottom.children[0] if _fp_script_bottom.children else ""
assert "const focusPosRaw = -1;" in _fp_js_bottom

# Top focus (0)
_fp_script_top = generate_card_stack_js(_fp_ids, _fp_btn, _fp_config, urls, focus_position=0)
_fp_js_top = _fp_script_top.children[0] if _fp_script_top.children else ""
assert "const focusPosRaw = 0;" in _fp_js_top

print("Focus position parameter tests passed!")

Focus position parameter tests passed!


In [ ]:
# Test extra_scripts injection
_reset_prefix_counter()
config2 = CardStackConfig()
ids2 = CardStackHtmlIds(prefix=config2.prefix)
btn2 = CardStackButtonIds(prefix=config2.prefix)

script2 = generate_card_stack_js(
    ids2, btn2, config2, urls,
    extra_scripts=("ns.customFunction = function() { console.log('custom'); };",)
)
js2 = script2.children[0] if script2.children else ""
assert "ns.customFunction" in js2
print("Extra scripts injection test passed!")

Extra scripts injection test passed!


In [ ]:
# Test with disable_scroll_in_modes
config3 = CardStackConfig(prefix="split-test", disable_scroll_in_modes=("split",))
ids3 = CardStackHtmlIds(prefix=config3.prefix)
btn3 = CardStackButtonIds(prefix=config3.prefix)

script3 = generate_card_stack_js(ids3, btn3, config3, urls)
js3 = script3.children[0] if script3.children else ""
assert "isScrollDisabled" in js3
assert "'split'" in js3
print("Scroll mode disabling in composed JS test passed!")

# Test with auto_visible_count=False excludes auto-adjust section
config4 = CardStackConfig(prefix="no-auto", auto_visible_count=False)
ids4 = CardStackHtmlIds(prefix=config4.prefix)
btn4 = CardStackButtonIds(prefix=config4.prefix)

script4 = generate_card_stack_js(ids4, btn4, config4, urls)
js4 = script4.children[0] if script4.children else ""
assert "Auto Visible Count Adjustment" not in js4
# assert "ns._runAutoAdjust" not in js4
# But handleCountChange and _isAutoMode are still in count management
assert "ns.handleCountChange" in js4
print("auto_visible_count=False exclusion test passed!")

Scroll mode disabling in composed JS test passed!
auto_visible_count=False exclusion test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()